In [1]:
from pathlib import Path

import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
from rlvr.utils.llm import get_default_model

env_id = "vf-musique-multi"
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
model = get_default_model()
temperature = 0.5
max_new_tokens = 1024

params = {
    "env_id": env_id,
    "datasets_str": datasets_str,
    "noise_rate": noise_rate,
    "retriever": retriever,
    "model": model,
    "temperature": temperature,
    "max_new_tokens": max_new_tokens,
}
params

{'env_id': 'vf-musique-multi',
 'datasets_str': 'bdsaglam/musique-mini,answerable,validation',
 'noise_rate': 1.0,
 'retriever': 'hybrid',
 'model': 'Qwen/Qwen2.5-7B-Instruct',
 'temperature': 0.5,
 'max_new_tokens': 1024}

In [3]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [4]:
# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id=env_id,
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
openai_client = AsyncOpenAI()
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-14 20:47:51 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Running 100 rollouts:   0%|          | 0/100 [00:00<?, ?it/s]Exception ignored in: <function AsyncHttpxClientWrapper.__del__ at 0x72ae90979080>
Traceback (most recent call last):
  File "/home/baris/repos/rlvr/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1355, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/home/baris/repos/rlvr/.venv/lib/python3.12/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'AsyncHttpxClientWrapper' object has no attribute '_state'
Exception ignored in: <function AsyncHttpxClientWrapper.__del__ at 0x72ae90979080>
Traceback (most recent call last):
  File "/home/baris/repos/rlvr/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1355, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/home/baris/repos/rlvr/.venv/lib/python3.12/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == Cl

[Trace(trace_id=tr-4ef2bcaed4f00c1ca49d9688be6397ea), Trace(trace_id=tr-a4ce46e3ced640a70a8f52fd3148b4e0), Trace(trace_id=tr-9b8132236399afbea8c410472fe05723), Trace(trace_id=tr-259a1d7c48a54e73e809ca6a7cdb10f5), Trace(trace_id=tr-cb667742e7d974d16c88f00f6c816c86), Trace(trace_id=tr-b422c8daaca1e599e49d6e0cfb1c872f), Trace(trace_id=tr-fc758723c104f7ce81c15b8dc82cb17f), Trace(trace_id=tr-920a5a7d47d1e0d7d8e16037cd290261), Trace(trace_id=tr-b46e2d0493c063a9b7bcdd9ac89b6489), Trace(trace_id=tr-2b3d63bcc1e64ff740b98526a63c47d8)]

In [5]:
ds = vf_env.make_dataset(results)
df = ds.to_pandas()

In [6]:
df.describe()

,reward,exact_match_reward,f1_reward,citation_reward,format_reward
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1.472786,0.200000,0.343286,0.635000,0.970000
std,0.911314,0.402015,0.403576,0.347102,0.171447
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.850000,0.000000,0.000000,0.500000,1.000000
50%,1.200000,0.000000,0.174242,0.500000,1.000000
75%,1.866667,0.000000,0.692308,1.000000,1.000000
max,3.200000,1.000000,1.000000,1.000000,1.000000


In [7]:
from datetime import datetime
import json

exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
exp_dir = Path(f"../outputs/musique-eval/{exp_id}")
exp_dir.mkdir(parents=True, exist_ok=True)

(exp_dir / "params.json").write_text(json.dumps(params, indent=2))
ds.to_json(exp_dir / "musique-eval-results.jsonl", orient="records", lines=True)
df.describe().to_csv(exp_dir / "scores.csv")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]